Засидевшись допоздна, студенты с кафедры электротехники заметили, что из-за перегоревших ламп в некоторых кабинетах воцарилась кромешная тьма. К счастью, у них в распоряжении есть целая коробка с различными лампами, каждая со своей мощностью и силой света. Чтобы студентам не пришлось перебирать все лампы, разработайте для них программу, позволяющую рассчитать освещённость произвольной точки кабинета. Считать источники света точечными и располагающимися в узлах прямоугольной сетки с шагом 1 метр. Естественным и отражённым светом пренебречь.
1. Лампа: излучаемая при включении сила света, потребляемая мощность, включение/выключение.
2. Светодиодная лампа: максимальные сила света и мощность для красного, зелёного и синего светодиода, их интенсивность, метод расчёта текущей силы света пропорциональной интенсивности красного, зелёного и синего светодиода.
3. Матрица источников света: источники света, метод расчёта освещённости в произвольных координатах (x, y, z) и мощности электропотребления.


In [149]:
import numpy as np
from abc import ABC, abstractmethod

In [150]:
class Lighting(ABC):
  '''
  Общий абстрактный класс для осветительных приборов
  '''

  __I: float
  __V: float

  def __init__(self, V, I):
    self.__V = float(V)
    self.__I = float(I)

  @abstractmethod
  def __call__(self):
    pass

  @abstractmethod
  def V(self):
    pass

  @abstractmethod
  def I(self):
    pass


class Lamp(Lighting):
  '''
  Обычная лампочка. На вход подаются аргументы: V - мощность, I - сила света, mode - включена или нет (True/False)
  '''

  __mode__: bool
  serial_number: int = 1

  def __init__(self, V=100., I=100., mode=True):
    super().__init__(V, I)
    self.__mode__ = mode
    self.__num = str(Lamp.serial_number)
    Lamp.serial_number+=1

  def __call__(self, state):
    '''
    Включение (on) и выключение (off)
    '''
    if state.lower() == 'on':
      self.__mode__ = True
      print("Now it's turned on")
    elif state.lower() == 'off':
      self.__mode__ = False
      print("Now it's turned off")

  def __bool__(self):
    return self.__mode__

  def __repr__(self):
    return self.__class__.__name__+self.__num

  @property
  def V(self):
    '''
    Потребляемая мощность
    '''
    return self._Lighting__V

  @property
  def I(self):
    '''
    Сила света
    '''
    return self._Lighting__I

  @property
  def mode(self):
    if self.__mode__:
      return 'on'
    else:
      return 'off'


class Diod(Lighting):
  '''
  Светодиод. На вход подаются: V - мощность, I - сила света, intensity - интенсивность
  '''

  __intensity__: float

  def __init__(self, V, I, intensity=1.0):
    super().__init__(V, I)
    self.__intensity__ = intensity

  def __call__(self, intensity):
    '''
    Изменение интенсивности свечения
    '''
    self.__intensity__ = float(intensity)
    print('Intensity set to', intensity)

  @property
  def intensity(self):
    return self.__intensity__

  @property
  def V(self):
    '''
    Текущая потребляемая мощность
    '''
    return (self._Lighting__V * self.__intensity__)

  @property
  def maxV(self):
    '''
    Максимально возможная потребляемая мощность
    '''
    return self._Lighting__V

  @property
  def I(self):
    '''
    Сила света
    '''
    return (self._Lighting__I * self.__intensity__)

  @property
  def maxI(self):
    '''
    Максимальная сила света
    '''
    return (self._Lighting__I)


class LED(Lamp):
  '''
  Светодиодная лампочка. На вход подаются светодиоды (r - красный, g - зелёный, b - синий) и mode - включена она или нет (True/False)
  '''

  red: Diod
  green: Diod
  blue: Diod

  def __init__(self, r = Diod(0.5, 0.2), g = Diod(0.5, 1.), b = Diod(0.5, 0.2), mode=True):
    self.red = r
    self.green = g
    self.blue = b
    super().__init__(V = self.V, I = self.I, mode = mode)

  @property
  def I(self):
    '''
    Текущая сила света
    '''
    return self.red.I+self.blue.I+self.green.I

  @property
  def V(self):
    '''
    Текущая потребляемая мощность
    '''
    return self.red.V+self.blue.V+self.green.V

  @property
  def maxV(self):
    '''
    Максимально возможная потребляемая мощность
    '''
    return self.red.maxV+self.blue.maxV+self.green.maxV

  @property
  def maxI(self):
    '''
    Максимально возможная сила света
    '''
    return self.red.maxI+self.blue.maxI+self.green.maxI

In [151]:
class Matrix:
  '''
  Матрица света. На вход:
  length, width - размеры комнаты вдоль и поперёк,
  height - высота потолков,
  lights - словарь источников света (вида объект-лампочка: её координаты)

  Пример lights: {Lamp(): (1, 1),
                  LED(r=Diod(2, 0.8)): (3, 3),
                  Lamp(220, 250, False): (0, 4)}

  Сетка расположения источников света - с шагом в один метр,
  начиная с нулевого метра от стены
  '''
  __ox: int
  __oy: int
  __oz: int
  lights: dict

  def __init__(self, length, width, height, lights):
    self.__ox = length
    self.__oy = width
    self.__oz = height
    self.lights = {}

    self.__map__ = np.array([[None]*(self.__oy+1)]*(self.__ox+1))
    self+lights

  def map(self):
    '''
    Карта расположения источников света
    '''
    for x in self.__map__:
      print(*x, sep='\t')

  def brightness(self, x, y, z):
    '''
    Рассчёт освещённости в точке (в канделах) по координатам
    '''
    bright = 0
    for l in self.lights.keys():
      if l:
        bright += l.I * (self.__oz - z) / (np.sqrt((self.lights[l][0]-x)**2+(self.lights[l][1]-y)**2+(self.__oz-z)**2))**3
    print('%.2f cd' %bright)

  @property
  def V(self):
    '''
    Мощность, затрачиваемая на комнату в данный момент
    (учитывается мощность только включённых ламп)
    '''
    v = 0
    for l in self.lights.keys():
      v += l.V
    return v

  def __sub__(self, coord):
    '''
    Удаление источника света в данной точке
    '''
    del self.lights[self.__map__[coord[0]][coord[1]]]
    self.__map__[coord[0]][coord[1]] = None

  def __add__(self, light):
    '''
    Добавление или перестановка источников света
    (принимает на вход словарь вида объект-лампочка: её координаты)
    '''
    for l in light.keys():
      if l in self.lights:
        self-(self.lights[l][0], self.lights[l][1])
      if self.__map__[light[l][0]][light[l][1]] is not None:
        del self.lights[self.__map__[light[l][0]][light[l][1]]]
      self.__map__[light[l][0]][light[l][1]] = l
      self.lights[l] = light[l]

  def __call__(self, x, y):
    '''
    Обращение к источнику света, находящемуся в данной точке
    '''
    if self.__map__[x][y] is None:
      print('No lamp here')
    return self.__map__[x][y]

Заведём набор лампочек и расположим его в комнате

In [152]:
lls = {Lamp(): (1, 1),
       LED(r=Diod(2, 0.8)): (3, 3),
       Lamp(220, 250, False): (0, 4)}

m = Matrix(3, 4, 2, lls)

Сейчас комната выглядит так:

In [153]:
m.map()

None	None	None	None	Lamp3
None	Lamp1	None	None	None
None	None	None	None	None
None	None	None	LED2	None


Лампочка номер 3 сейчас выключена:

In [154]:
m(0, 4).mode

'off'

Поэтому освещение в комнате потребляет следующую мощность:

In [155]:
m.V

323.0

В таком случае освещённость пола в дальнем углу составляет:

In [156]:
m.brightness(0, 0, 0)

13.65 cd


Если включить лампочку номер три, станет светлее:

In [157]:
m(0,4)('on')
m.brightness(0, 0, 0)

Now it's turned on
19.24 cd


Поменяем набор источников света

Переставим светодиодную лампочку ближе к нашему углу, заменим лампочку номер один на новую светодиодную, добавим лампочку в самый угол, уберём лампочку номер 3

In [158]:
m+{m(3,3):(0,1)}

In [159]:
m+{LED():(1,1)}

In [160]:
m+{Lamp(220, 400):(0,0)}

In [161]:
m-(0, 4)

In [162]:
m.map()

Lamp5	LED2	None	None	None
None	LED4	None	None	None
None	None	None	None	None
None	None	None	None	None


Теперь в нашем углу светлее и мощности на освещение комнаты тратится меньше

In [163]:
m.brightness(0, 0, 0)
m.V

100.55 cd


224.5

In [164]:
m.lights

{LED2: (0, 1), LED4: (1, 1), Lamp5: (0, 0)}